In [ ]:
import sys
sys.path.append('/usr/local/lib/python3.8/dist-packages/torch/__init__.py')
sys.path.append('/lustrehome/emanueleamato/.local/lib/python3.11/site-packages')
import keras
from keras import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
import tensorflow as tf
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation


In [ ]:
Controllo dimensioni immagine 

In [ ]:
# Carica l'immagine utilizzando Keras

img_path = '###Load Image###' 
img = image.load_img(img_path)

# Converti l'immagine in un array NumPy
img_array = image.img_to_array(img)

# Controlla il numero di canali dell'immagine
num_channels = img_array.shape[-1]  # Assumendo che l'immagine sia nel formato (H, W, C) dopo la conversione
print("Numero di canali dell'immagine nel dataset:", num_channels)

In [ ]:

def SimpleCNN():
    model = Sequential()
    
    # Passo 1: Convoluzione -> Attivazione -> Dropout
    model.add(Conv2D(8, (3, 3), strides=(1, 1), padding='same', input_shape=(64, 64, 3)))
    model.add(Activation('relu'))
    model.add(Dropout(0.3))
    
    # Passo 2: Convoluzione -> Attivazione -> Pooling
    model.add(Conv2D(16, (3, 3), strides=(1, 1), padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
    
    # Passo 3: Convoluzione -> Pooling
    model.add(Conv2D(32, (3, 3), strides=(1, 1), padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
    
    # Appiattimento (flatten)
    model.add(Flatten())
    
    # Passo 4: Fully Connected Layer -> Attivazione
    model.add(Dense(32))
    model.add(Activation('relu'))
    
    # Passo 5: Fully Connected Layer (output layer)
    model.add(Dense(2))  # Numero di classi
    model.add(Activation('softmax'))  # Softmax per output di classificazione
    
    return model


# Addestro

In [ ]:
# Iperparametri
batch_size = 32
learning_rate = 0.001
num_epochs = 5

# Carica il dataset di immagini senza trasformazioni
train_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
    '/lustrehome/emanueleamato/ViT_Test/Train',
    batch_size=batch_size,
    class_mode='sparse'  
)

test_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
    '/lustrehome/emanueleamato/ViT_Test/Test',
    batch_size=batch_size,
    class_mode='sparse'
)

# Verifica i dispositivi disponibili
devices = tf.config.experimental.list_physical_devices('GPU')
print("Devices:", devices)

# Verifica se sono disponibili dispositivi GPU
#if devices:
 #   tf.config.experimental.set_visible_devices(devices[0], 'GPU')

# Definisci il modello
model = SimpleCNN()

# Compila il modello
model.compile(optimizer=Adam(lr=learning_rate),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Callback per il salvataggio del modello durante l'addestramento
checkpoint = ModelCheckpoint("best_model.h5", monitor="val_accuracy", save_best_only=True)

# Addestra il modello
history = model.fit(train_generator,
                    steps_per_epoch=len(train_generator),
                    epochs=num_epochs,
                    validation_data=test_generator,
                    validation_steps=len(test_generator),
                    callbacks=[checkpoint],
                    verbose=1)